In [11]:
!git lfs install
!git clone https://huggingface.co/hexgrad/Kokoro-82M
%cd Kokoro-82M
!apt-get -qq -y install espeak-ng > /dev/null 2>&1
!pip install -q phonemizer torch transformers scipy munch

Git LFS initialized.
Cloning into 'Kokoro-82M'...
remote: Enumerating objects: 140, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 140 (delta 63), reused 0 (delta 0), pack-reused 4 (from 1)
Receiving objects: 100% (140/140), 60.13 KiB | 12.03 MiB/s, done.
Resolving deltas: 100% (63/63), done.
Filtering content: 100% (17/17), 820.18 MiB | 54.05 MiB/s, done.
/content/Kokoro-82M
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.4/213.4 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [12]:
import torch
import soundfile as sf
import librosa


from time import time
from IPython.display import display, Audio

from kokoro import generate
from models import build_model
from transformers import WhisperProcessor, WhisperForConditionalGeneration

In [48]:
stt_processor = WhisperProcessor.from_pretrained("openai/whisper-tiny.en")
stt_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny.en")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.94k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

In [61]:
voices = ['af', 'af_bella', 'af_sarah', 'am_adam', 'am_michael', 'bf_emma', 'bf_isabella', 'bm_george', 'bm_lewis', 'af_nicole', 'af_sky',]
text = "Greetings, adventurer, how may I be of help to you?"

In [58]:
def convert24to16(audio):
  audio_resampled = librosa.resample(audio, orig_sr=24000, target_sr=16000)
  audio_filename = f'generated_{name}.wav'
  sf.write(audio_filename, audio_resampled, 16000)
  audio_data, sr = librosa.load(audio_filename, sr=16000)
  audio_input = stt_processor(audio_data, return_tensors="pt", sampling_rate=16000)
  return audio_input

In [64]:
for name in voices:
    VOICEPACK = torch.load(f'voices/{name}.pt', weights_only=True).to('cpu')
    start = time()
    audio, out_ps = generate(MODEL, text, VOICEPACK, lang=name[0])
    print(f"{time()-start:.2f} seconds: Generated audio for {name} on cpu")
    # mb there is a way to avoid 24000 -> 16000
    audio_input = convert24to16(audio)
    recognize = time()
    with torch.no_grad():
      predicted_ids = stt_model.generate(**audio_input)

        # Decode the predicted ids into text
      transcription = stt_processor.decode(predicted_ids[0], skip_special_tokens=True)

    print(f"{time()-recognize:.2f} seconds on cpu: Transcription for {name}: {transcription}")
    display(Audio(data=audio, rate=24000, autoplay=True))

7.12 seconds: Generated audio for af on cpu
1.44 seconds on cpu: Transcription for af:  Greetings, adventurer. How may I be of help to you?


5.37 seconds: Generated audio for af_bella on cpu
1.25 seconds on cpu: Transcription for af_bella:  Greetings, adventurer. How may I be of help to you?


7.61 seconds: Generated audio for af_sarah on cpu
1.25 seconds on cpu: Transcription for af_sarah:  Greetings, adventurer. How may I be of help to you?


4.80 seconds: Generated audio for am_adam on cpu
1.27 seconds on cpu: Transcription for am_adam:  Greetings Adventurer, how may I be of help to you?


7.57 seconds: Generated audio for am_michael on cpu
1.28 seconds on cpu: Transcription for am_michael:  Greetings, adventurer. How may I be of help to you?


...

7.25 seconds: Generated audio for bf_isabella on cpu
1.27 seconds on cpu: Transcription for bf_isabella:  Greetings, adventurer. How may I be of help to you?


5.58 seconds: Generated audio for bm_george on cpu
1.27 seconds on cpu: Transcription for bm_george:  Greetings, adventurer. How may I be of help to you?


7.36 seconds: Generated audio for bm_lewis on cpu
1.30 seconds on cpu: Transcription for bm_lewis:  Greetings, Adventurer, how may I be of help to you?


7.22 seconds: Generated audio for af_nicole on cpu
2.00 seconds on cpu: Transcription for af_nicole:  Greetings, adventurer. How may I be of help to you?


5.70 seconds: Generated audio for af_sky on cpu
1.27 seconds on cpu: Transcription for af_sky:  Greetings, adventurer. How may I be of help to you?
